In [5]:
# standard imports
import joblib
import os
import re
import warnings
import json
import time
import argparse
import numpy as np
import openai
import tiktoken
import langchain

import pandas as pd
import concurrent.futures
from langchain.schema import Document
from langchain.callbacks import get_openai_callback
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
import random
# external imports
import faiss
from datetime import datetime
from sentence_transformers import SentenceTransformer
from azureml.core.model import Model
from azureml.core import Workspace
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [6]:
from azureml.core import Workspace, Dataset
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws,name="test-assist-code-search-faiss-index-test")
faiss_model_azure=model.download(exist_ok = True)

In [7]:
print(faiss_model)

Model(workspace=Workspace.create(name='sv-wus2-subprod-etg-dev-ws', subscription_id='a0776237-28a9-41f7-88f3-a1e65953748f', resource_group='sv-wus2-subprod-etg-dev-rg'), name=test-assist-code-search-faiss-index-test, id=test-assist-code-search-faiss-index-test:1, version=1, tags={}, properties={})


In [8]:
def restructure_document_object(self, matching_docs):
    filtered_documents = []
    for row in matching_docs:
        input_doc_content = row.page_content + \
            "\n\n" + row.metadata.get('unit_test_cases')
        document_obj = {"input": row.page_content , "output":row.metadata.get('unit_test_cases')}
        filtered_documents.append(document_obj)
    return filtered_documents


In [ ]:
def few_shot_prompting_template(self,examples):
    example_template = """
    Input:
        Task:
        write unit test cases in Jasmine Framework for below servicenow script, create various test scenarios using multiple data points

        Instructions:
        - understand the code snippet below and write unit test cases
        - consider every scenario for unit testing
        - only write code
        - no comments and explanations for the written code

        code snippet:
        ####

        {input}

        ####

        Example output format:
        //comments
        <code>
        </code>

    Output:
        {output}
    """
    example_prompt = PromptTemplate(
        input_variables=["input", "output"], template=example_template
    )
    
    prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        suffix="",
        input_variables=["input"],
    )

    examples_retreived = prompt.format(input="")
    
    return examples_retreived



In [ ]:

if len(matching_docs) == 0:
    system_message = "Think of Yourself as a QA Engineer and tester. You will be provided with Java script source code as input and you need to generate unit test cases in Jasmine framework as output "
    value = [{"role": "System", "content": quote(system_message)},
             {"role": "user", "content": quote(prompt)}]
    


In [9]:
examples = [
    {
        "input": "Who lived longer, Muhammad Ali or Alan Turing?",
        "output": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
"""
    },
    {
        "input": "When was the founder of craigslist born?",
        "output": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
"""
    },
    {
        "input": "Who was the maternal grandfather of George Washington?",
        "output": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
"""
    },
    {
        "input": "Are both the directors of Jaws and Casino Royale from the same country?",
        "output": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
"""
    }
]

In [ ]:
system_message = "Think of Yourself as a QA Engineer and tester. You will be provided with Java script source code as input and you need to generate unit test cases in Jasmine framework as output "
value = [{"role": "System", "content": quote(system_message)},
         {"role": "System", "content": quote(examples_retreived)},
         {"role": "user", "content": quote(prompt)}]

In [ ]:

class Inference:

    def __init__(self, model):
        self.model_path = model
        self.embeddings = SentenceTransformerEmbeddings(
            model_name="multi-qa-MiniLM-L6-cos-v1")
        self.db = FAISS.load_local(self.model_path, self.embeddings,allow_dangerous_deserialization=True)

        self.api_key_16k = os.environ.get('ULLM_TEST_ASSIST_GPT_35_TURBO_API_KEY')
        self.api_key = os.environ.get('ULLM_TEST_ASSIST_GPT_35_TURBO_16K_API_KEY')
        self.url = os.environ.get('ULLM_TEST_ASSIST_GPT_35_TURBO_URL')

    def gpt_35_turbo_chat_completion(self,prompt):
        #self.allowSelfSignedHttps(True)
        print(prompt)
        value = [{"role": "user", "content": quote(prompt)}]
        value_unquote = [{"role": "user", "content": prompt}]
        turbo_data = {
            "messages": value,
            'temperature': 0,
            'top_p': 0.1
        }

        turbo_data_unquote = {
            "messages": value_unquote,
            'temperature': 0,
            'top_p': 0.1
        }
        print(str(turbo_data_unquote))
        #print(str(turbo_data))
        turbo_body = str.encode(json.dumps(turbo_data))
        url = self.url
        api_key = self.api_key
        if not api_key:
            raise Exception("A key should be provided to invoke the endpoint")

        headers = {'Content-Type': 'application/json','api-key': api_key}

        #req = urllib.request.Request(url, turbo_body, headers)
        result =""
        prompt_response=""
        tokens_count = 0
        total_cost = 0.0
        model =""
        status =""
        status_code = 0
        error_message=""

        try:
            #req = urllib.request.Request(url, turbo_body, headers)
            response = requests.post(url=url,headers=headers,data=turbo_body)
            status_code = response.status_code
            response_json = response.json()
            print(response_json)
            prompt_response = response_json['choices'][0]['message']['content']
            print(prompt_response)
            tokens_count = response_json['usage']['total_tokens']
            total_cost = (int(response_json['usage']['prompt_tokens'])/1000)*0.0005 + (int(response_json['usage']['completion_tokens'])/1000)*0.0015
            total_cost = '{:.8f}'.format(total_cost)
            model = response_json['model']
            status = "success"

        except requests.exceptions.RequestException as error:  # This is the correct syntax
            
            status_code = error.errno
            status = "failed"
            print(error)
            raise(error)

        return prompt_response,tokens_count,total_cost,model,status,status_code,error_message


    def gpt_35_turbo_16k_chat_completion(self,prompt):
        #self.allowSelfSignedHttps(True)
        print(prompt)
        value = [{"role": "user", "content": quote(prompt)}]
        value_unquote = [{"role": "user", "content": prompt}]
        turbo_data = {
            "messages": value,
            'temperature': 0,
            'top_p': 0.1
        }

        turbo_data_unquote = {
            "messages": value_unquote,
            'temperature': 0,
            'top_p': 0.1
        }
        print(str(turbo_data_unquote))
        #print(str(turbo_data))
        turbo_body = str.encode(json.dumps(turbo_data))
        url = self.url
        api_key = self.api_key_16k
        if not api_key:
            raise Exception("A key should be provided to invoke the endpoint")

        headers = {'Content-Type': 'application/json','api-key': api_key}

        #req = urllib.request.Request(url, turbo_body, headers)
        result =""
        prompt_response=""
        tokens_count = 0
        total_cost = 0.0
        model =""
        status =""
        status_code = 0
        error_message=""

        try:
            #req = urllib.request.Request(url, turbo_body, headers)
            response = requests.post(url=url,headers=headers,data=turbo_body)
            status_code = response.status_code
            response_json = response.json()
            print(response_json)
            prompt_response = response_json['choices'][0]['message']['content']
            print(prompt_response)
            tokens_count = response_json['usage']['total_tokens']
            total_cost = (int(response_json['usage']['prompt_tokens'])/1000)*0.0005 + (int(response_json['usage']['completion_tokens'])/1000)*0.0015
            total_cost = '{:.8f}'.format(total_cost)
            model = response_json['model']
            status = "success"

        except requests.exceptions.RequestException as error:  # This is the correct syntax
            
            status_code = error.errno
            status = "failed"
            print(error)
            raise(error)

        return prompt_response,tokens_count,total_cost,model,status,status_code,error_message

    def num_tokens_from_string(self, string: str, encoding_name: str) -> int:
        encoding = tiktoken.get_encoding(encoding_name)
        num_tokens = len(encoding.encode(string))
        return num_tokens

    def code_clean(self,code):
        code = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b', f"{{{'email'}}}", code)
        code = re.sub(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', f"{{{'phone'}}}", code)
        code = re.sub(r'\b\d{3}[-]?\d{2}[-]?\d{4}\b', f"{{{'ssn '}}}", code)
        code = re.sub(r'\b\d{3}[ ]?\d{2}[ ]?\d{4}\b', f"{{{'ssn'}}}", code)
        code = re.sub(r'\b(?:\d{1,3}\.){3}\d{1,3}\b', f"{{{'ip address'}}}", code)
        code = re.sub(r'//.*', '', code)
        code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
        return code

    # Without global variables
    def split_func(self,code):
        code = self.code_clean(code)
        classes=code.split('\n')
        classlist=[]
        scriptlist=[]
        cla=''
        for i in range(len(classes)):
            #dividing classes
            if len(re.findall(r'Class\.create\(\)',classes[i]))>0:
                classlist.append(cla)
                cla=''
                cla+=classes[i]+'\n'
            else:
                cla+=classes[i]+'\n'
            if(i==len(classes)-1):
                classlist.append(cla)
                cla=''
        classlist.pop(0)
        final_classlist=[]
        for i in range(len(classlist)):   
            #counting number of functions in every class
            fu=re.findall(r':\s*function\s*\(',classlist[i])
            if(len(fu)>1):
                lines=classlist[i].split('\n')
                #getting all the script before starting first function
                prefix=''
                ind=2
                for j in range(len(lines)):
                    if len(re.findall(r':\s*function\s*\(',lines[j]))>0:
                        ind=j
                        break
                    else:
                        prefix+='\n'+lines[j]
                #not considering global variables
                temp_prefix=prefix.split('\n')
                prefix2=temp_prefix[1]+'\n'+temp_prefix[-1]
            
                #exluding class defining code lines
                nlines=lines[ind:-2]
                func=[]
                temp=''
                br=0
                comment=False
                for j in range(len(nlines)):
                    if len(re.findall(r':\s*function\s*\(',nlines[j]))>0 and '{' in nlines[j]:
                        br+=1
                        temp+='\n'+nlines[j]
                        continue
                    elif len(re.findall(r':\s*function\s*\(',nlines[j]))>0 and '{' not in nlines[j]:
                        temp+='\n'+nlines[j]
                        continue
                    if '{' in nlines[j]:
                        br+=1
                    if '}' in nlines[j]:
                        br-=1
                    if br==0 and '}' in nlines[j]:
                        temp+='\n'+nlines[j]
                        #adding class defining code prior to function
                        temp=prefix2+temp+'\n'+'};'
                        #appending all functions snippets to list
                        func.append(temp)
                        temp=''
                    if len(temp)>0:
                        temp+='\n'+nlines[j]
                    
                #replacing that paticular class string with list of function strings
                for script in func:
                    final_classlist.append(script)
            else:
                final_classlist.append(classlist[i])

        return final_classlist

    def gpt35turbo_large_script_generate(self,code):
        token_count = self.num_tokens_from_string(code,"cl100k_base")
        no_list = int(np.ceil(token_count/4000))
        list_func = self.split_func(code)
        # using list comprehension 
        num_rec = int(np.ceil(len(list_func)/no_list))
        list_func_split = [list_func[i:i + num_rec] for i in range(0, len(list_func), num_rec)]
        final_list=[]
        for i in list_func_split:
            if self.num_tokens_from_string('\n'.join(i),"cl100k_base")>6000:
                token_count_s = self.num_tokens_from_string('\n'.join(i),"cl100k_base")
                no_list_s = int(np.ceil(token_count_s/4000))
                list_func_s = self.split_func('\n'.join(i))
                # using list comprehension 
                num_rec_s = int(np.ceil(len(list_func_s)/no_list_s))
                list_func_split_s = [list_func_s[i:i + num_rec_s] for i in range(0, len(list_func_s), num_rec_s)]
                for j in list_func_split_s:
                    final_list.append('\n'.join(j))
            else:
                final_list.append('\n'.join(i))

        response,time_taken,token_count,total_cost,model_used,error,error_type,azure_time_taken,self_calculated_token_size = self.parallel_api_call(final_list)
        
        return response,time_taken,token_count,total_cost,model_used,error,error_type,azure_time_taken,self_calculated_token_size

    def restructure_document_object(self, matching_docs):
        filtered_documents = []
        for row in matching_docs:
            input_doc_content = row.page_content + \
                "\n\n" + row.metadata.get('unit_test_cases')
            document_obj = Document(page_content=input_doc_content)
            filtered_documents.append(document_obj)
        return filtered_documents

    def gpt35turbo_langchain_generate(self, code):

        start_time = datetime.now()
        response = None
        total_cost = ""
        tokens_count = ""
        model_used = ""
        error = ""
        error_type = ""
        azure_start_time = ""
        azure_end_time = ""
        input_prompt_size=0
        matching_docs=[]
        if len(code) <= 5:
            response = None
            error = "length of source code is too short to generate a response"
            error_type = "not_enough_tokens"

        elif len(code) > 5:

            code = self.code_clean(code)
            prompt_template = PromptTemplate.from_template(
            """
            Task:
            write unit test cases in Jasmine Framework for below servicenow script, create various test scenarios using multiple data points

            Instructions:
            - understand the code snippet below and write unit test cases
            - consider every scenario for unit testing
            - only write code
            - no comments and explanations for the written code

            code snippet:
            ####

            {code}

            ####

            Example output format:
            //comments
            <code>
            </code>

            """)
            
            
            prompt = prompt_template.format(code = code)

            input_prompt_size = self.num_tokens_from_string(
                prompt, "cl100k_base")
            print("input prompt size: " + str(input_prompt_size))

            if input_prompt_size < 1500:
                model_used = 'gpt-35-turbo'

                retriever = self.db.as_retriever(
                    search_type="similarity_score_threshold", search_kwargs={
                        "score_threshold": 0.5, "k": 1})
                matching_docs = retriever.get_relevant_documents(code)

                # if len(matching_docs) == 0:
                #    retriever = self.db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5,"k": 1})
                #    matching_docs = retriever.get_relevant_documents(code)

            elif input_prompt_size >= 1500 and input_prompt_size < 6000:
                model_used = 'gpt-35-turbo-16k'

                retriever = self.db.as_retriever(
                    search_type="similarity_score_threshold", search_kwargs={
                        "score_threshold": 0.5, "k": 1})
                matching_docs = retriever.get_relevant_documents(code)

            else:
                model_used = 'gpt-35-turbo-16k'
                matching_docs = []

            azure_start_time = datetime.now()

            if len(matching_docs) == 0:
                system_message = "Think of Yourself as a QA Engineer and tester. You will be provided with Java script source code as input and you need to generate unit test cases in Jasmine framework as output "
                value = [{"role": "System", "content": quote(system_message)},
                         {"role": "user", "content": quote(prompt)}]
                prompt_response,tokens_count,total_cost,model,status,status_code,error_message = self.gpt_35_turbo_16k_chat_completion(value)
            else:
                matching_docs = self.restructure_document_object(matching_docs)
                chain = load_qa_chain(self.gpt_35_turbo_16k_chat_completion, chain_type="stuff")
                response,tokens_count,total_cost,model,status,status_code,error  = chain.run(
                    input_documents=matching_docs, question=prompt)
        azure_end_time = datetime.now()

        end_time = datetime.now()
        azure_time_taken = round(
            (azure_end_time - azure_start_time).total_seconds(),
            2)  # type: ignore
        time_taken = round((end_time - start_time).total_seconds(), 2)
        return response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken


